<a href="https://colab.research.google.com/github/maniparimi/langgraph/blob/main/langgraphtools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langraph langsmith langchain langchain_grok langchain_community

In [ ]:
from typing import Annotated, Dict

In [ ]:
from langchain_community.utilities.utilities import RCFAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import RCQueryRun, WikipediaQueryRun

In [ ]:
rcf_wrapper = RCFAPIWrapper(top_k_results=1, document_content_chars_max=300)
wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=1, document_content_chars_max=300)

rcf_tool = RCQueryRun(wrapper=rcf_wrapper)
wikipedia_tool = WikipediaQueryRun(wrapper=wikipedia_wrapper)

In [ ]:
pip install rcf wikipedia

In [ ]:
response = wikipedia_tool.invoke("Who is a famous actor?")
print(response)

response = rcf_tool.invoke("Attention is all you need")
print(response)

In [ ]:
tools = [wikipedia_tool]

In [ ]:
from langraph.message import add_message
from langraph import State

class ChatState(State):
    messages: Annotated[list, Dict]

In [ ]:
from langraph import State, Graph, Start, End

In [ ]:
graph_builder = Graph.Builder(state=ChatState)

In [ ]:
from langchain_grok import ChatGrok
from google.colab import user_data

grok_api_key = user_data.get('grok_api_key')

llm = ChatGrok(grok_api_key=grok_api_key, model_name='gamma-29b')

In [ ]:


llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
class Chatbot(ChatState):
    def run(self):
        return llm_with_tools.invoke(self.messages)

In [ ]:
graph_builder.add_node('chatbot', Chatbot)
graph_builder.add_edge('start', 'chatbot')

from langraph import ToolNode, ToolConditions

tool_node = ToolNode(tools=tools)
graph_builder.add_node('tools', tool_node)

graph_builder.add_conditional_edges('chatbot', 'tools', ToolConditions())
graph_builder.add_edge('tools', 'chatbot')
graph_builder.add_edge('chatbot', 'end')

In [ ]:
graph = graph_builder.compile()

from IPython.display import display

display(graph.visualize())

In [ ]:
user_input = "Hi there, my name is John."

events = graph.stream(messages=[{'role': 'user', 'content': user_input}], stream_mode='values')

for event in events:
    print(event.messages[-1].pretty_print())